In [ ]:
!pip install obspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# read, format and classify data from original mseed files

Ong, Giani, Nielsen

Modified for multiple stations by Dewsnap

**mount Google Drive as a disk to access files and data**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/DISS_FOLDER/New_Data"

/content/drive/MyDrive/DISS_FOLDER/New_Data


### **INITIALISATION**

In [ ]:
choice = 'old' # old=norm on all 3 components together. new2=no norm. new=norm on individual components,

numStations = 3

years = [2014,2015,2016,2017,2018,2019,2020,2021,2022] #add more when more folders are finished
#2013 has no events with full data
#INCN is offline for most of 2012-2013 (alongside other stations missing entries/axes)
#2012 all entries have anomalies.

stationNames = ["MAJO.IU","ERM.II","INCN.IU"]

for i in range(numStations):
  print("Taking events from station "+stationNames[i])


Taking events from station MAJO.IU
Taking events from station ERM.II
Taking events from station INCN.IU


**load libraries for importing and formatting the input data:**
---



In [ ]:

from obspy import read as readobs
from obspy import read_inventory
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
import random
import math

**Import the "original" data from mseed files, format them:**

In [ ]:
# Read all datafiles in folder and store them in "events":

events = []
K=0
for year in years:
    print("\n=============")
    print(year)
    yearPath = './'+str(year)

    eventNames = os.listdir(yearPath)
    print(str(len(eventNames))+" events")
    print("=============\n")

    K = K+len(eventNames)
    for event in eventNames:
      #i+=1
      #print("Event" + str(i))
      print(event)

      #for i in range(numStations):
      events.append([readobs("./"+str(year)+"/"+event+"/"+station+".mseed") for station in stationNames])
      #events.append(read("./Data/seeddata32/TEST8.mseed"))



# Form selectedevents, extract the 3 channels as data, and store them in "eventdata"
eventdata = []
for event in events: #for each event's data
    stationData = []
    for station in event: # for each station
      flag = False
      for i in range(3): #for each axis (x,y,z), deals with multiple stations e.g for 2 stations gives a list of the form [x1,y1,z1,x2,y2,z2]
        stationData.append(station[i].data)
        #print(len(station[i].data))
        if len(station[i].data)!=1440000:
          flag=True
          #print("Flagged!")
    if flag ==False:
      eventdata.append(stationData)
    
#Concatenate channels for each event
eventdataconcat = []
for stationAxis in eventdata:
    data_concat_TEMP=0
    for i in range(3*numStations):
      st_TEMP = np.expand_dims(stationAxis[i],axis=-1)

      if isinstance(data_concat_TEMP,int):
        data_concat_TEMP=st_TEMP
      else:
        data_concat_TEMP = np.c_[data_concat_TEMP,st_TEMP]

    #sta = np.expand_dims(event[0], axis=-1)
    #stb = np.expand_dims(event[1], axis=-1)
    #stc = np.expand_dims(event[2], axis=-1)
    eventdataconcat.append(data_concat_TEMP)

#print(eventdataconcat[0][0])


2014
5 events

2014_E4.996941
2014_E6.125507
2014_E15.958592
2014_E7.767260
2014_E14.417538

2015
7 events

2015_E2.41843
2015_E7.512251
2015_E5.34634
2015_E13.637238
2015_E10.502950
2015_E14.799543
2015_E15.257925

2016
4 events

2016_E2.15144
2016_E3.823861
2016_E8.149332
2016_E9.871807

2017
4 events

2017_E1.727519
2017_E2.174189
2017_E4.721584
2017_E3.654490

2018
6 events

2018_E4.590218
2018_E5.447225
2018_E3.920499
2018_E6.658486
2018_E9.425817
2018_E7.37024

2019
3 events

2019_E1.463352
2019_E8.941687
2019_E10.518755

2020
1 events

2020_E2.774661

2021
3 events

2021_E1
2021_E4
2021_E11

2022
3 events

2022_E5
2022_E6
2022_E7


The structure of each event in "eventdataconcat" is (for 2 stations): <br>
array([  <br>
[x1, y1, z1, x2, y2, z2], # time 1 <br>
[x1, y1, z1, x2, y2, z2], # time 2 <br>
..... <br>
[x1, y1, z1, x2, y2, z2]  # time n <br>
]) <br>
(n is the total number of time samples)

### Extract first and last 40k samples and class as noise and precusros, respectivley

In [ ]:
def create_precursors(X1, last = 40000):
    precurstr = X1[-last:] 
    return(precurstr)
    
def create_noise(X1, first = 0, second = 40000):
    precurstr = X1[first:second]
       
    return(precurstr)

#Select the final 40000 time steps from each event (all 3 channels) and label as 'precursors' 
#Select the first 40000 time steps from each event (all 3 channels) and label as 'noise' 

precursors = []
noise = []
for event in eventdataconcat:
    precursors.append(create_precursors(event,40000))
    
for event in eventdataconcat:
    noise.append(create_noise(event, 0, 40000))

### Make windows of length "window_length" from the selected 'noise' and 'precursor' data 
The windows overlap of 650 timesteps
With this overlap, each precursor window of 40000 samples produces 37 "normpre"v windows of 16384 samples. 
Same for noise windows.

In [ ]:
window_length=16384

def make_windows(X1, sample_stride = 650):
    X2 = []
    for i in range(len(X1)-window_length):        
        if i % sample_stride == 0:
               X2.append(X1[i:i+window_length])    
    return(X2)
   
precursor_windows = []
for event in precursors:
    precursor_windows.append(make_windows(event, 650))
    
noise_windows = []
for event in noise:
    noise_windows.append(make_windows(event, 650))

### Normalisation of signal
Three different options for normalisation fuction: 
- new 
- new2
- old

In previous work (Ong et al.) normalisation_old was used. 

In [ ]:
def normalise_new(X1): # Here each 3 individual component of each window is mean stripped and normalised
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        mea=np.mean(data,axis=0).reshape(-1, 1)
        values = (data.T-mea).T
        values = values / np.linalg.norm(values,axis=0,ord=np.Inf)
        X2.append(values)
    return X2

def normalise_new2(X1): # not normalise, not strip mean
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        values = data
        X2.append(values)
    return X2

def normalise_old(X1): ## Here the 3 components are mean stripped and normalised together, results strange but working for the CNN:
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        values = data - np.mean(data)
        values = values / np.linalg.norm(values)
        X2.append(values)
    return X2


normpre = []
normnoise = []

if choice == 'new2':
  for event in precursor_windows:    normpre.append(normalise_new2(event))
  for event in noise_windows:    normnoise.append(normalise_new2(event))
elif choice == 'old':
  for event in precursor_windows:    normpre.append(normalise_old(event))
  for event in noise_windows:    normnoise.append(normalise_old(event))
elif choice == 'new':
  for event in precursor_windows:    normpre.append(normalise_new(event))
  for event in noise_windows:    normnoise.append(normalise_new(event))


### Select events for train and test datasets. These events were randomly selected.

In [ ]:
trainlist = []

trainlist = [0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 17, 19, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
testlist = [8, 9, 16, 18, 20, 22, 24]

if len(trainlist)==0:

  K=len(eventdataconcat)

  print(K)

  trainlist = random.sample(range(K),round(4*K/5))

  temp = set(trainlist)
  temp2 = set(range(K))

  testlist = list(temp2-temp)

print(sorted(trainlist))
print(len(trainlist))
print()
print(sorted(testlist))
print(len(testlist))






[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 17, 19, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
28

[8, 9, 16, 18, 20, 22, 24]
7


## output data to files for further use by the CNN

In [ ]:
print(choice)
pickle.dump( normpre, open( "./Data/normpre_"+choice+".p", "wb" ) ) # pickle file with all precursor windows
pickle.dump( normnoise, open( "./Data/normnoise_"+choice+".p", "wb" ) ) # pickle file with all noise windows
pickle.dump( trainlist, open( "./Data/trainlist.p", "wb" ) ) # pickel file with the labels of events in train data
pickle.dump( testlist, open( "./Data/testlist.p", "wb" ) ) # pickle file with labels of events in test data


old
